In [32]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from pylab import mpl
import seaborn as sns
import numpy as np

In [38]:
filepath = '../preprocessed_supermarket.xlsx'
df = pd.read_excel(filepath)
df = df[['制造商','产品名称','产品描述','类别','子类别','地区','省/自治区','城市','细分','订单日期','折扣','数量']]
print(df)

             制造商   产品名称 产品描述    类别  子类别  地区 省/自治区  城市   细分       订单日期   折扣  数量
0        Fiskars     剪刀   蓝色  办公用品   用品  华东    浙江  杭州   公司 2016-04-27  0.4   2
1      GlobeWeis   搭扣信封   红色  办公用品   信封  西南    四川  内江  消费者 2016-06-15  0.0   2
2       Cardinal  孔加固材料   回收  办公用品  装订机  西南    四川  内江  消费者 2016-06-15  0.4   2
3       Kleencut    开信刀   工业  办公用品   用品  华东    江苏  镇江   公司 2016-12-09  0.4   4
4     KitchenAid    搅拌机   黑色  办公用品   器具  中南    广东  汕头  消费者 2015-05-31  0.0   3
...          ...    ...  ...   ...  ...  ..   ...  ..  ...        ...  ...  ..
9930     SanDisk    记忆卡   实惠    技术   配件  华东    浙江  义乌  消费者 2016-05-28  0.4   3
9931       Eldon      盘   蓝色  办公用品  收纳具  华东    浙江  义乌  消费者 2016-05-28  0.0   2
9932       Safco    折叠椅   红色    家具   椅子  华东    浙江  义乌  消费者 2016-05-28  0.4   1
9933          罗技    路由器   耐用    技术   配件  华东    浙江  义乌  消费者 2016-05-28  0.4   7
9934    Breville     冰箱   白色  办公用品   器具  华东    浙江  温州   公司 2013-12-02  0.4   5

[9935 rows x 12 columns]


In [39]:
filepath = '../products/products_info.xlsx'
df2 = pd.read_excel(filepath)
df2 = df2[['产品名称','类别','子类别','制造商','单价']]
df = df.merge(df2).drop_duplicates().reset_index().drop(columns=['index'])
print(df)

          制造商 产品名称 产品描述    类别 子类别  地区 省/自治区  城市    细分       订单日期    折扣  数量  \
0     Fiskars   剪刀   蓝色  办公用品  用品  华东    浙江  杭州    公司 2016-04-27  0.40   2   
1     Fiskars   剪刀   蓝色  办公用品  用品  华东    浙江  温岭    公司 2015-06-07  0.40   6   
2     Fiskars   剪刀  锯齿状  办公用品  用品  西南    云南  开远    公司 2015-08-30  0.00   3   
3     Fiskars   剪刀  锯齿状  办公用品  用品  西南    云南  昆明    公司 2015-12-13  0.00   5   
4     Fiskars   剪刀   蓝色  办公用品  用品  华东    山东  潍坊   消费者 2016-05-28  0.00   5   
...       ...  ...  ...   ...  ..  ..   ...  ..   ...        ...   ...  ..   
9930    Bevis   木桌   黑色    家具  桌子  西南    四川  成都   消费者 2016-08-05  0.60   1   
9931    Lesro   木桌   组装    家具  桌子  中南    广东  珠海    公司 2016-12-11  0.25   3   
9932    Lesro   木桌   白色    家具  桌子  中南    广东  恩城  小型企业 2014-06-20  0.25   5   
9933    Lesro   木桌   白色    家具  桌子  中南    湖北  随州    公司 2015-11-18  0.60   2   
9934    Bevis  电脑桌   黑色    家具  桌子  华北    天津  天津    公司 2016-09-04  0.25  13   

               单价  
0      104.340789  
1      104.340789  
2  

## 构造负类样本

In [40]:
from datetime import timedelta
time_threshold = timedelta(days=5)
synthetic_data = []

# 遍历原始数据
for index, row in df.iterrows():
    # 选择相同产品名称和产品描述、制造商不同、且订单日期相差5天以内的数据
    similar_rows = df[
        (df['产品名称'] == row['产品名称']) &
        (df['产品描述'] == row['产品描述']) &
        (df['制造商'] != row['制造商']) &
        (abs(df['订单日期'] - row['订单日期']) <= time_threshold)
    ]
    
    if similar_rows.shape[0] !=0:
        # 针对符合条件的数据，创建新的合成数据
        for _, similar_row in similar_rows.iterrows():
            synthetic_row = row.copy()  # 复制原始数据行
            synthetic_row['制造商'] = similar_row['制造商']  # 替换制造商
            synthetic_row['订单日期'] = similar_row['订单日期']  # 替换订单日期
            synthetic_row['单价'] = similar_row['单价']  # 替换价格
            synthetic_row['折扣'] = similar_row['折扣']  # 替换折扣
            
            synthetic_data.append(synthetic_row.values.tolist())

synthetic_df = pd.DataFrame(synthetic_data, columns=df.columns)
print(synthetic_df.head())

        制造商 产品名称 产品描述    类别 子类别  地区 省/自治区   城市   细分       订单日期   折扣  数量  \
0  Kleencut   剪刀    钢  办公用品  用品  华北    河北  秦皇岛  消费者 2016-01-09  0.0   3   
1  Stiletto   剪刀    钢  办公用品  用品  华北    河北  秦皇岛  消费者 2016-01-02  0.0   3   
2      Acme   剪刀  锯齿状  办公用品  用品  华东    山东   即墨   公司 2016-11-09  0.4   5   
3  Stiletto   剪刀  锯齿状  办公用品  用品  华北    河北  秦皇岛   公司 2013-11-09  0.0   1   
4  Kleencut   剪刀   工业  办公用品  用品  中南    广西   钦州   公司 2015-05-31  0.4   3   

           单价  
0   99.191047  
1  106.528319  
2  117.379087  
3  106.528319  
4   99.191047  


In [27]:
print(synthetic_df.shape)

(1816, 12)


## 增加新列并合并数据集

In [41]:
synthetic_df['是否购买'] = 0
df['是否购买'] = 1
df = pd.concat([synthetic_df, df], axis=0, ignore_index=True)  # ignore_index 重新索引
print(df)

            制造商 产品名称 产品描述    类别 子类别  地区 省/自治区   城市    细分       订单日期    折扣  数量  \
0      Kleencut   剪刀    钢  办公用品  用品  华北    河北  秦皇岛   消费者 2016-01-09  0.00   3   
1      Stiletto   剪刀    钢  办公用品  用品  华北    河北  秦皇岛   消费者 2016-01-02  0.00   3   
2          Acme   剪刀  锯齿状  办公用品  用品  华东    山东   即墨    公司 2016-11-09  0.40   5   
3      Stiletto   剪刀  锯齿状  办公用品  用品  华北    河北  秦皇岛    公司 2013-11-09  0.00   1   
4      Kleencut   剪刀   工业  办公用品  用品  中南    广西   钦州    公司 2015-05-31  0.40   3   
...         ...  ...  ...   ...  ..  ..   ...  ...   ...        ...   ...  ..   
11748     Bevis   木桌   黑色    家具  桌子  西南    四川   成都   消费者 2016-08-05  0.60   1   
11749     Lesro   木桌   组装    家具  桌子  中南    广东   珠海    公司 2016-12-11  0.25   3   
11750     Lesro   木桌   白色    家具  桌子  中南    广东   恩城  小型企业 2014-06-20  0.25   5   
11751     Lesro   木桌   白色    家具  桌子  中南    湖北   随州    公司 2015-11-18  0.60   2   
11752     Bevis  电脑桌   黑色    家具  桌子  华北    天津   天津    公司 2016-09-04  0.25  13   

                单价  是否购买  


In [42]:
df.to_excel("probability_info.xlsx", index=False)